In [1]:
import pandas as pd
import numpy as np
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier

In [2]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.width',None)
pd.set_option('display.max_colwidth',None)

In [3]:
## setting work directory

In [4]:
os.chdir("U:\Propensity_Model\Model")

In [5]:
### Loading the required files

In [6]:
pickle_in = open("Final_Data_.pickle","rb")

Final_Data = pickle.load(pickle_in)

pickle_in.close()

In [7]:
### Split data into Train, Test and Validation

In [8]:
x = Final_Data.drop('AUDIT_STATUS',axis=1)
y = pd.DataFrame(Final_Data['AUDIT_STATUS'])
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [9]:
## Checking the Selection %

In [11]:
print(y_train['AUDIT_STATUS'].value_counts()/len(y_train)*100) #1.77
print(y_test['AUDIT_STATUS'].value_counts()/len(y_test)*100)   #1.88

0    98.226685
1     1.773315
Name: AUDIT_STATUS, dtype: float64
0    98.119926
1     1.880074
Name: AUDIT_STATUS, dtype: float64


In [12]:
####Data PreProcessing (numeric columns of train_data)

##Separating out Numeric columns from train data

In [13]:
train_num = x_train.select_dtypes(include='number')

In [14]:
##PreProcessing numeric data

In [15]:
sc = StandardScaler()
train_num_process = pd.DataFrame(sc.fit_transform(train_num))
train_num_process.columns = train_num.columns
train_num_process.index = train_num.index

In [16]:
### Combining the processed data to train_data

In [17]:
col_drop = train_num.columns
x_train = x_train.drop(col_drop,axis=1)

In [18]:
## Combining the PreProcessed data to train_data

In [19]:
x_train = pd.concat([x_train,train_num_process],axis=1)

In [20]:
x_train = x_train.astype({'REV_MISSING':'Int64','REV_0636':'Int64','REV_0301':'Int64','REV_0300':'Int64','REV_0250':'Int64','REV_0305':'Int64','REV_0637':'Int64'
                            ,'REV_0260':'Int64','REV_0450':'Int64','REV_0306':'Int64','REV_0258':'Int64','REV_0420':'Int64','REV_0272':'Int64','REV_0730':'Int64',
                            'REV_0762':'Int64','REV_OTHERS':'Int64','CPT_MISSING':'Int64','CPT_83735':'Int64','CPT_80048':'Int64','CPT_96372':'Int64','CPT_96375':'Int64'
                            ,'CPT_93005':'Int64','CPT_84484':'Int64','CPT_G0378':'Int64','CPT_A9270':'Int64','CPT_85025':'Int64','CPT_80053':'Int64','CPT_36415':'Int64'
                            ,'CPT_J3490':'Int64','CPT_OTHERS':'Int64', 'REJ_MISSING':'Int64', 'REJ_0DC':'Int64', 'REJ_64V':'Int64','REJ_88':'Int64', 'REJ_0DC29T':'Int64'
                            ,'REJ_0DC46R':'Int64', 'REJ_0IREJ_04@':'Int64', 'REJ_04)':'Int64','REJ_05D':'Int64', 'REJ_26P':'Int64','REJ_97':'Int64', 'REJ_OTHERS':'Int64'})

In [19]:
## performing one-hot-encoding on categorical features

In [21]:
ctgrl_col = pd.DataFrame(x_train[['PR_NPAR','LINE_ITEM','OVR','LOB','NTWK','PRV_SPC','POT','TOB','PAR_CD','CAS_CAUSE','VCH','MBR_ST','PRIM_DX','ADMIT_DX','CLI','SUSP']])
ctgrl_cols = pd.get_dummies(ctgrl_col)

x_train = x_train.drop(ctgrl_col.columns,axis=1)
x_train = pd.concat([x_train,ctgrl_cols],axis=1)

In [22]:
### Data balancing using SMOTE

In [23]:
sm = SMOTE(sampling_strategy=0.43)

In [24]:
x_sm,y_sm = sm.fit_resample(x_train.values,y_train.values.ravel())

In [25]:
x_sm = pd.DataFrame(x_sm)
y_sm = pd.DataFrame(y_sm)

In [26]:
x_sm.columns = x_train.columns
y_sm.columns = y_train.columns

In [27]:
y_sm.value_counts(normalize=True)*100

AUDIT_STATUS
0               69.931045
1               30.068955
dtype: float64

In [28]:
y_train.value_counts(normalize=True)*100

AUDIT_STATUS
0               98.226685
1                1.773315
dtype: float64

In [29]:
####################################################
####################################################
####################################################

### Model Building

In [30]:
KNN_v1 = KNeighborsClassifier()


In [31]:
data = {
    'n_neighbors':np.arange(1,25),
    'weights':['uniform','distance'],
    'p':[1,2],
    'metric':['minkowski']
}

In [32]:
from sklearn.model_selection import RandomizedSearchCV
gscv = RandomizedSearchCV(KNN_v1,data,cv=10,n_jobs=-1)
gscv.fit(x_sm.values,y_sm.values.ravel())

RandomizedSearchCV(cv=10, estimator=KNeighborsClassifier(), n_jobs=-1,
                   param_distributions={'metric': ['minkowski'],
                                        'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24]),
                                        'p': [1, 2],
                                        'weights': ['uniform', 'distance']})

In [33]:
gscv.best_params_

{'weights': 'distance', 'p': 1, 'n_neighbors': 3, 'metric': 'minkowski'}

In [34]:
gscv.best_score_

0.9810629034621584

In [35]:
KNN_v1 = KNeighborsClassifier(weights='distance', p=1, n_neighbors=3, metric='minkowski')
KNN_v1.fit(x_sm.values,y_sm.values.ravel())

KNeighborsClassifier(n_neighbors=3, p=1, weights='distance')

In [36]:
#####################################

####Data PreProcessing (numeric columns of test_data)

##Separating out Numeric columns from test data

In [37]:
test_num = x_test.select_dtypes(include='number')

In [38]:
##PreProcessing numeric data

In [39]:
test_num_process = pd.DataFrame(sc.transform(test_num))
test_num_process.columns = test_num.columns
test_num_process.index = test_num.index

In [40]:
###########
### Combining the processed data to test_data

In [41]:
col_drop = test_num.columns
x_test = x_test.drop(col_drop,axis=1)

In [42]:
## Combining the Preprocessed data to test_data

In [43]:
x_test = pd.concat([x_test,test_num_process],axis=1)

In [44]:
x_test = x_test.astype({'REV_MISSING':'Int64','REV_0636':'Int64','REV_0301':'Int64','REV_0300':'Int64','REV_0250':'Int64','REV_0305':'Int64','REV_0637':'Int64'
                            ,'REV_0260':'Int64','REV_0450':'Int64','REV_0306':'Int64','REV_0258':'Int64','REV_0420':'Int64','REV_0272':'Int64','REV_0730':'Int64',
                            'REV_0762':'Int64','REV_OTHERS':'Int64','CPT_MISSING':'Int64','CPT_83735':'Int64','CPT_80048':'Int64','CPT_96372':'Int64','CPT_96375':'Int64'
                            ,'CPT_93005':'Int64','CPT_84484':'Int64','CPT_G0378':'Int64','CPT_A9270':'Int64','CPT_85025':'Int64','CPT_80053':'Int64','CPT_36415':'Int64'
                            ,'CPT_J3490':'Int64','CPT_OTHERS':'Int64', 'REJ_MISSING':'Int64', 'REJ_0DC':'Int64', 'REJ_64V':'Int64','REJ_88':'Int64', 'REJ_0DC29T':'Int64'
                            ,'REJ_0DC46R':'Int64', 'REJ_0IREJ_04@':'Int64', 'REJ_04)':'Int64','REJ_05D':'Int64', 'REJ_26P':'Int64','REJ_97':'Int64', 'REJ_OTHERS':'Int64'})

In [45]:
##one-hot-encoding on categorical features

In [46]:
ctgrl_col = pd.DataFrame(x_test[['PR_NPAR','LINE_ITEM','OVR','LOB','NTWK','PRV_SPC','POT','TOB','PAR_CD','CAS_CAUSE','VCH','MBR_ST','PRIM_DX','ADMIT_DX','CLI','SUSP']])
ctgrl_cols = pd.get_dummies(ctgrl_col)

x_test = x_test.drop(ctgrl_col.columns,axis=1)
x_test = pd.concat([x_test,ctgrl_cols],axis=1)

In [47]:
##################
### Predicting on the test data

In [48]:
##RF_v1

In [52]:
y_pred = KNN_v1.predict(x_test)
y_pred = (KNN_v1.predict_proba(x_test)[:,1]>0.2).astype('int').astype('str')
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

print(classification_report(y_test,y_pred))

C:\Users\KPA2877\.conda\envs\test1\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
C:\Users\KPA2877\.conda\envs\test1\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


0.9438826378008831
[[6607  282]
 [ 112   20]]
              precision    recall  f1-score   support

           0       0.98      0.96      0.97      6889
           1       0.07      0.15      0.09       132

    accuracy                           0.94      7021
   macro avg       0.52      0.56      0.53      7021
weighted avg       0.97      0.94      0.95      7021



In [50]:
y_pred = KNN_v1.predict(x_sm)
y_pred = (KNN_v1.predict_proba(x_sm)[:,1]>0.5).astype('int').astype('str')
print(accuracy_score(y_sm,y_pred))
print(confusion_matrix(y_sm,y_pred))

print(classification_report(y_sm,y_pred))

C:\Users\KPA2877\.conda\envs\test1\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
C:\Users\KPA2877\.conda\envs\test1\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


0.9996197333062922
[[27585     0]
 [   15 11846]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     27585
           1       1.00      1.00      1.00     11861

    accuracy                           1.00     39446
   macro avg       1.00      1.00      1.00     39446
weighted avg       1.00      1.00      1.00     39446

